### FEI and S&P+ Web Scraper

In [13]:
### cloest_fei_dictionary(team, closest n teams, past k years):
### return dictionary: key:team, value: list of year or years if same team closest in past k

In [18]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
from collections import Counter
import csv
import re

In [19]:
def extract_team_name(x):
    match = re.search('\>(.*)\<', x)
    if match:
        found = match.group(1)
    return found

def extract_def_fei(x):
    match = re.search('\>(.*)\<', x)
    if match:
        found = match.group(1)
    return found

def extract_def_sp(x):
    match = re.search('[0-9]*\.[0-9]*', x)
    if match:
        found = match.group(0)
    return found


In [20]:
teams = []
defsp = []
year = []

for i in np.arange(2014, 2018):

    url = "https://www.footballoutsiders.com/stats/ncaadef" + str(i)
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page)
    
    raw_team_name = []
    raw_all_def_sp = []

    for tr in soup.find_all('tr')[1:]:
        tds = tr.find_all('td')
        team = tds[0]
        def_sp = tds[1]
        raw_team_name.append(team)
        raw_all_def_sp.append(def_sp)
    
    raw_str_team_name = [str(x) for x in raw_team_name]
    raw_str_def_sp = [str(x) for x in raw_all_def_sp]

    teams.append([extract_team_name(x) for x in raw_str_team_name])
    defsp.append([extract_def_sp(x) for x in raw_str_def_sp])
    year.append([i]*len(raw_str_team_name))
    
teams = sum(teams, [])
defsp = sum(defsp, [])
year = sum(year, [])

s_and_p_data = pd.DataFrame({'Team' : teams, 'Def. S&P+' : defsp, 'Year' : year})
drop = s_and_p_data.iloc[:, 0] == "."
s_and_p_data = s_and_p_data[~drop]
s_and_p_data["Def. S&P+"] = [float(x) for x in s_and_p_data["Def. S&P+"]]
s_and_p_data = s_and_p_data.sort_values("Def. S&P+").drop_duplicates()

In [21]:
s_and_p_data.head()

,Def. S&P+,Team,Year
663,5.9,Alabama,2016
531,7.7,Michigan,2016
398,9.0,Alabama,2015
134,12.2,Ole Miss,2014
133,12.2,Clemson,2014


In [82]:
def n_closest_def_sp(team_sp, n):
    closest_data = s_and_p_data.iloc[(s_and_p_data["Def. S&P+"] - team_sp).abs().argsort()[:n]]
    percent_diff = (abs(np.array(closest_data.loc[:, "Def. S&P+"]) - team_sp)/team_sp)*100
    keys = closest_data.Team
    values = closest_data.Year
    return list(zip(keys, values, percent_diff))

In [83]:
n_closest_def_sp(13, 10)

[('Florida', 2016, 2.3076923076923128),
 ('LSU', 2016, 3.0769230769230793),
 ('Boston College', 2015, 4.6153846153846132),
 ('Michigan', 2015, 4.6153846153846132),
 ('Ohio State', 2016, 4.6153846153846132),
 ('Ole Miss', 2014, 6.1538461538461586),
 ('Clemson', 2014, 6.1538461538461586),
 ('Clemson', 2016, 6.1538461538461586),
 ('Alabama', 2017, 6.1538461538461586),
 ('Clemson', 2017, 6.923076923076926)]

In [8]:
### return list of tuples instead of dictionary:
    ### easier to iterate over
    ### sorted by closest match no alaphabetical

In [9]:
teams = []
dfei = []
year = []


for i in np.arange(2017, 2018):

    url = "https://www.footballoutsiders.com/stats/feidef" + str(i)
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page)
    
    raw_team_name = []
    raw_all_def_fei = []

    for tr in soup.find_all('tr')[1:]:
        tds = tr.find_all('td')
        team = tds[1]
        def_fei = tds[3]
        raw_team_name.append(team)
        raw_all_def_fei.append(def_fei)
    
    raw_str_team_name = [str(x) for x in raw_team_name]
    raw_str_def_fei = [str(x) for x in raw_all_def_fei]

    teams.append([extract_team_name(x) for x in raw_str_team_name])
    dfei.append([extract_def_fei(x) for x in raw_str_def_fei])
    year.append([i]*len(raw_str_team_name))
    
teams = sum(teams, [])
dfei = sum(dfei, [])
year = sum(year, [])

pd.DataFrame({'Team' : teams, 'DFEI' : dfei, 'Year' : year})

,DFEI,Team,Year
0,.83,Clemson,2017
1,.88,Alabama,2017
2,.93,Wisconsin,2017
3,.95,Auburn,2017
4,.98,Georgia,2017
5,1.17,Texas,2017
6,1.24,Ohio State,2017
7,1.37,Mississippi State,2017
8,1.37,Michigan,2017
9,1.37,Washington,2017


In [10]:
def_fei_data = []

for x in np.arange(2017, 2018):
    url = "https://www.footballoutsiders.com/stats/feidef" + str(x)
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page)
    
    team_name = []
    all_def_fei = []

    for tr in soup.find_all('tr')[1:]:
        tds = tr.find_all('td')
        team = tds[1]
        def_fei = tds[3]
        team_name.append(team)
        all_def_fei.append(def_fei)
    
    def_fei_data.append("")

In [11]:
[str(x) for x in def_fei_data[0]]

[]

In [12]:
r""

''